### Books recommender system using clustering | Collaborative based

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
books = pd.read_csv('data/books_data/books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


C:\Users\hp\AppData\Local\Temp\ipykernel_5664\1381155323.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/books_data/books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


In [3]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [4]:
books.shape

(271360, 8)

In [5]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [6]:
books = books.filter(items=['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L'])
books.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [7]:
books.rename(columns={"Book-Title": "title",
                       "Book-Author": "author",
                       "Year-Of-Publication": "year",
                       "Publisher": "publisher",
                       "Image-URL-L": "image_url"}, inplace=True)
books.head(1)

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...


In [8]:
users = pd.read_csv('data/books_data/users.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv('data/books_data/ratings.csv', sep=';', encoding='latin-1', on_bad_lines='skip')

In [9]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [10]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [11]:
users.rename(columns={"User-ID": "user_id",
                       "Location": "location",
                       "Age": "age"}, inplace=True)
users.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [12]:
ratings.rename(columns={"User-ID": "user_id",
                       "Book-Rating": "rating"}, inplace=True)
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [13]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [14]:
x = ratings['user_id'].value_counts() > 200
x[x].shape

(899,)

In [15]:
y = x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [16]:
ratings = ratings[ratings['user_id'].isin(y)]
ratings.shape

(526356, 3)

In [17]:
rating_with_books = ratings.merge(books, on='ISBN')
rating_with_books.shape

(487671, 8)

In [18]:
rating_with_books.head(3)

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...


In [19]:
num_ratings = rating_with_books.groupby('title')['rating'].count().reset_index()
num_ratings.rename(columns={'rating': 'num_ratings'}, inplace=True)
num_ratings.head()

,title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [20]:
final_rating = rating_with_books.merge(num_ratings, on='title')

In [21]:
final_rating.head(3)
final_rating.shape

(487671, 9)

In [22]:
final_rating = final_rating[final_rating['num_ratings'] >= 50]
final_rating.shape


(61853, 9)

In [23]:
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)
final_rating.shape

(59850, 9)